In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('./preprocess/filter3.csv')

In [2]:
messages = df['Message'].values.tolist()
text_processed = messages[:10]

file_path = './preprocess/language.txt'
keyword_list = []
with open(file_path, 'r') as file:
    for line in file:
        word = line.strip()  # Remove leading/trailing whitespaces and newline characters
        keyword_list.append(word)

print(keyword_list)


['php:', 'php engineer', 'PHP Developer', 'java:', 'Java Developer', 'java engineer', 'Spring', 'javascript:', 'javascript engineer', 'JS Developer', 'Typescript', 'AI Engineer:', 'AI Developer', 'Dầu Nhật:', 'Dầu Nhật Bản', 'Người gõ máy:', 'Người gõ máy tính', 'Nodejs:', 'Nodejs Developer', 'Nodejs Engineer', 'ExpressJS/Lookback/Koa/NestJS/MeteorJS framework.', 'ExpressJS', 'Lookback', 'Koa', 'NestJS', 'MeteorJS', 'BrSE:', 'MIDDLE BRSE', 'SENIOR BRSE', 'Brse PM', 'Brse TL', 'C#:', '.NET', '.Net', '.Net Developer', 'C# Developer', 'C# Engineer', 'C++:', 'C++ Developer', 'C++ Engineer', 'Middle/Senior C++', 'network:', 'automation test:', 'Automation Tester', 'frontend:', 'html', 'css', 'html/css', 'html/css/javascript', 'UI/UX:', 'UI/UX DESIGNER', 'RUBY:', 'SENIOR BACKEND RUBY', 'BACKEND RUBY', 'python:', 'Python Developer', 'Python Engineer', 'Python', 'Linux:', 'Linux Developer', 'Linux Engineer', 'Operator:', 'Operator', 'operator', 'ReactJS:', 'ReactJS Developer', 'ReactJS Enginee

In [14]:
import gensim
from gensim.models import Word2Vec
import matplotlib.pyplot as plt

path = './models/word2vec_model.bin'
w2v_model = Word2Vec.load(path)

In [15]:
def caculate_score(sentence):
    arr_score = []
    if isinstance(sentence, float):
        return arr_score
    words = sentence.split()
    
    for word in words:
        scores = []
        for word2 in keyword_list:
            if word in w2v_model.wv.key_to_index and word2 in w2v_model.wv.key_to_index:
                score = w2v_model.wv.similarity(word, word2)
            else:
                score = 0
            scores.append(score)
        arr_score.append(scores)
    print('arr_score', arr_score)
    max_index = np.max(arr_score, axis=0)
    print('max_index', max_index)
    return max_index

df['Score'] = df['Message'][:100].apply(caculate_score)
print(df.head())
df['Score'].to_csv('./preprocess/score.csv', index=True, header=['Score'])

arr_score [[0, 0, 0, 0, 0, 0, 0.095757626, 0, 0, 0, -0.043770496, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.06526086, 0.0075008697, 0.005312111, 0.068529725, -0.050254244, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.019812822, -0.06685923, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0045275986], [0, 0, 0, 0, 0, 0, -0.061486892, 0, 0, 0, -0.056024704, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.008175481, 0.14019871, 0.093686774, -0.10600808, 0.093364134, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.25812, -0.26639363, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.16607353], [0, 0, 0, 0, 0, 0, -0.067536786, 0, 0, 0, -0.16964583, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.0396851, 0.04978854, -0.026610676, -0.06583991, -0.11670416, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.16615023, -0.0728727, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.03526496], [0, 0, 0, 0, 0, 0, -0.14634538, 0, 0, 0, -0.22060812, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.024193147, 0.029016979, -4.4973567e-05, -0.048187785, -0.11925648, 0, 0, 

In [16]:
def setLabelsMax(x):
    result = [(index, value) for index, value in enumerate(x) if value != 0.0]
    result = sorted(result, key=lambda x: x[1], reverse=True)
    result = [(keyword_list[x[0]], x[1]) for x in result]
    return result
df['Labels'] = df['Score'][:100].apply(setLabelsMax)
df['Message_Short'] = df['Message'].str[:300]

df_predict = df[['Labels', 'Message_Short']][:100]
print('df_predict', df_predict.head())
df_predict.to_csv('./preprocess/predict.csv', index=True, header=['Labels', 'Message_Short'])

df_predict                                               Labels  \
0  [(Lookback, 0.3551449775695801), (Koa, 0.34544...   
1  [(Python, 0.31398266553878784), (Lookback, 0.2...   
2  [(html, 0.19105291366577148), (Spring, 0.18226...   
3  [(Typescript, 0.2219592183828354), (Python, 0....   
4  [(Spring, 0.19635766744613647), (Python, 0.170...   

                                       Message_Short  
0  Năm 2023 đánh dấu nhiều tiến bộ và các xu hướn...  
1  🌟CẦN BỔ SUNG 6 BẠN PHỤ Các công việc về Mãng C...  
2  Dầu Nhật IKO Chỉ Bạn Miễn Phí Cách Tăng Thời G...  
3  (HẢI CHÂU) FRONTEND (VUEJS) LEADER 🚀UPTO 45M G...  
4  **T.ì.m người gõ văn bản thuê.**\n\n**300k/ 1 ...  
